In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mickey-donald/test_data(13).csv
/kaggle/input/mickey-donald/sample_output(16).csv
/kaggle/input/mickey-donald/train_data(14).csv


In [2]:
train = pd.read_csv('/kaggle/input/mickey-donald/train_data(14).csv')
test = pd.read_csv('/kaggle/input/mickey-donald/test_data(13).csv')

In [3]:
train

,datapointID,sample,dialect,category
0,516,$NE$ $NE$ fin suplinite folosului notificările...,2,5
1,135,"asigurator săgeată, predicție cover înfrângă a...",2,2
2,568,notificările asigurator importanţei $NE$ $NE$ ...,1,5
3,1180,"$NE$ $NE$ privatizată robotic beijingul ""$NE$ ...",1,5
4,871,$NE$ $NE$ 1 întocmească saptămână fundaţie mod...,2,3
...,...,...,...,...
1147,1225,"""$NE$ irite adolescentă ilicită dictatorial ""m...",1,3
1148,658,ntechne indispensabile $NE$ $NE$ ajutata testa...,1,5
1149,578,$NE$ $NE$ nerăbdători stadiului $NE$ #$NE$ – $...,2,3
1150,391,$NE$ $NE$ $NE$ soţii tendinţe - prindea $NE$ $...,1,3


In [4]:
train['dialect'].unique()

array([2, 1])

In [5]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def dialect(value):
    if value == 1:
        return 1
    else:
        return 0
def cleantext(text):
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = text.replace('$NE$', '')    
    text = text.lower()

    return text

def prep(df, mode = 'train'):
    df = df.copy()
    df.bfill(inplace = True)
    df['sample'] = df['sample'].apply(cleantext)
    if mode == 'train':
        df['dialect'] = df['dialect'].apply(dialect)

    return df

In [7]:
train = prep(train)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(train['sample'])

In [9]:
import xgboost as xgb
import lightgbm as lgb

xgb = xgb.XGBClassifier()

lgbm = lgb.LGBMClassifier(
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=500,
    random_state=42
)

In [10]:
x_train = vectorized
y_train = train['dialect']

xgb.fit(x_train,y_train)
lgbm.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 521, number of negative: 631
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29987
[LightGBM] [Info] Number of data points in the train set: 1152, number of used features: 1184
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.452257 -> initscore=-0.191556
[LightGBM] [Info] Start training from score -0.191556


LGBMClassifier(learning_rate=0.05, n_estimators=500, random_state=42)

In [11]:
test = prep(test, 'test')
x_test = vectorizer.transform(test['sample'])

In [12]:
preds = xgb.predict(x_test)
preds2 = lgbm.predict(x_test)


for i in range(0, len(preds2)):
    if preds2[i]==1 or preds2[i] == True:
        preds2[i] = 1
    else:
        preds2[i] = 2

In [13]:
for i in range(0, len(preds)):
    if preds[i]==1 or preds[i] == True:
        preds[i] = 1
    else:
        preds[i] = 2

In [14]:
preds

array([2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2,
       1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2,
       1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1,
       1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2,
       1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2,
       1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1,
       2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1,
       2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1,
       1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1,
       2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1,

In [15]:
submission = pd.DataFrame({
    'subtaskID' : (['1'] * len(test)) + (['2'] * len(test)),
    'datapointID' : test['datapointID'].tolist() + test['datapointID'].tolist(),
    'answer' : preds2.tolist() + preds2.tolist()
})

In [16]:
submission.to_csv('submission.csv', index = False)

In [17]:
#t:nitro nlp

#problema propusa de ales set de date